In [2]:
import traceback
import urllib.request
import requests
import importlib
import boto3
import pandas
import numpy


In [ ]:
#
# Cell to automatically Test accessing the webservice basic interface
# 

test_ecg_data = '1.0,1.0,1.0, 1.0,  1.3'

heartconmon.webservice.start()

try:
    contents = urllib.request.urlopen("http://127.0.0.1:8080").read()
    r = requests.post("http://127.0.0.1:8080", data={'ecgdata': test_ecg_data, 'freq': 202})
    # TODO Verify results from the GET/POST messages
except:
    traceback.print_exc()
finally:
    heartconmon.webservice.stop()


In [ ]:
#
# Cell to start up the webservice interface and manually test it
#
heartconmon.webservice.start()

In [ ]:
#
# Cell to stop the webservice interface
#
heartconmon.webservice.stop()

In [ ]:
#
# This cell can be used to verify connectivity to AWS.
#



# credentials should be loaded from disk
aws_client = boto3.client('dynamodb', region_name='us-east-1')

print(aws_client.describe_table(TableName='Session'))
print(aws_client.get_item(TableName='Session', Key={'sessionId' : { 'S' : 'mytestsession'}, 'deviceId' : { 'S' : '5'}})['Item'])

In [20]:
import matplotlib.pyplot as plt
print(len(test_new_session['beats']))
print(test_new_session['beats'])
# plt.plot(pandas.read_csv('./heartconmon/physionet_test/Cardiomyopathy/0550_re.csv', header=None).values[:,1][0:1000])
# plt.show()
# for beat in test_new_session['beats'][0:50]:
#     plt.plot(beat['norm_data'])
#     plt.ylabel('some numbers')
#     plt.show()


0
[]


In [21]:
#
# Cell to manually test the session processor.
# 
# Data structure:
# Session {
#    'sessionId' : str<uuid>,
#    'time' : str<timestamp>,
#    'frequency' : float  in Hertz
#    'data' : [float float ...],
#    'status' : string in set('active', 'inactive', 'unknown'),
#    'beats' : [
#        'start' : int,
#        'stop' : int,
#        'value' : [float float ...],
#        'conditions' : [
#            {
#                'name' : str,
#                'conf' : float
#            },
#        ]
#    ]
# }
#
import heartconmon.webservice
importlib.reload(heartconmon.webservice)

test_new_session = {
   'sessionId' : 'asdasdasdasdasdasdasdasd',
   'frequency' : 200.0,
   'data' : pandas.read_csv('./heartconmon/physionet_test/Cardiomyopathy/0550_re.csv', header=None).values[:,1],
   'status' : 'active',
   'beats' : []
}

heartconmon.webservice.test_record = test_new_session

count = 0
def session_term_hook():
    global count
    count += 1
    if count > 5:
        heartconmon.webservice.test_record['status'] = 'inactive'
        print('terminating test session')


try:
    sp = heartconmon.webservice.SessionProcessor('asdasdasdasdasdasdasdasd', session_term_hook)
    sp.handle_session()
except Exception as e:
    import traceback
    traceback.print_exc()
    print(e)
    print('stuff happened')
finally:
    pass

# print(test_new_session)

Error retrieving record. Using test record
Error retrieving record. Using test record
'type'
stuff happened


Traceback (most recent call last):
  File "<ipython-input-21-1349c25a0ba4>", line 48, in <module>
    sp.handle_session()
  File "C:\Users\jeffl\Documents\GitHub\cmsc495_7980\heartconmon\webservice.py", line 417, in handle_session
    self._save_record()
  File "C:\Users\jeffl\Documents\GitHub\cmsc495_7980\heartconmon\webservice.py", line 389, in _save_record
    beat['conditions'] = [{'name' : _class_to_name(beat['type']), 'confidence' : beat['conf']}]
KeyError: 'type'
